In [1]:
import os
import pandas as pd
from cleantext import clean
import json
import glob
from pprint import pprint
import datetime
import re  
import nltk  
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm
nltk.download()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cheng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
ethereum_path = "../data/eth_all.csv"

ethereum = pd.read_csv(ethereum_path)

ethereum.head(50)

,Unnamed: 0,Unnamed: 0.1,Datetime,Text\r
0,0,0,2022-05-02 23:59:59+00:00,【123战法早8点播报-2022.5.3】\n#BTC 日线：空，小时线：多\n#ETH ...
1,1,1,2022-05-02 23:59:45+00:00,https://t.co/MeCEt7jfgS @DNC will be approving...
2,2,2,2022-05-02 23:59:15+00:00,4 hour top movers report #blockchain #crypto #...
3,3,3,2022-05-02 23:59:12+00:00,#Bitcoin Last Price $38512 #BTC 🚀\nDaily Indic...
4,4,4,2022-05-02 23:59:02+00:00,$RickMortyDoxx $RMD #RMDOXX #ETH #Ethereum #Ai...
5,5,5,2022-05-02 23:58:55+00:00,Bored Apes crashed #Ethereum. Transaction fees...
6,6,6,2022-05-02 23:58:42+00:00,tokenviz: 🌕 Biggest #Cryptos 1-Day Volume\n\n#...
7,7,7,2022-05-02 23:58:31+00:00,#VTHOUSDT Bull Alert!\n\n5X Volume\nPrice: ...
8,8,8,2022-05-02 23:58:24+00:00,"The $ETH price is at $2,857.38 right now.\n🔴 C..."
9,9,9,2022-05-02 23:58:23+00:00,Hi @TheBinanceNFT \n\nI'm holding https://t.co...


In [8]:
df = ethereum.filter(['Text\r','Datetime'])[0:150016]

df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))


,Text\r,Datetime
0,1238-2022.5.3\n#BTC \n#ETH \nBTC38000ET...,2022-05-02 23:59:59+00:00
1,https://t.co/MeCEt7jfgS @DNC will be approving...,2022-05-02 23:59:45+00:00
2,4 hour top movers report #blockchain #crypto #...,2022-05-02 23:59:15+00:00
3,#Bitcoin Last Price $38512 #BTC \nDaily Indica...,2022-05-02 23:59:12+00:00
4,$RickMortyDoxx $RMD #RMDOXX #ETH #Ethereum #Ai...,2022-05-02 23:59:02+00:00
...,...,...
150010,good morning to the crowd\n\n...and have a nic...,2022-05-15 05:41:39+00:00
150011,Curated Coin Metrics every day \n\n#ETH | 2...,2022-05-15 05:41:27+00:00
150012,Paris No. 3 (1/1) .10eth\n\nI BELONG IN (TRAV...,2022-05-15 05:41:14+00:00
150013,Woman by MaiPrisin\n\nhttps://t.co/GKcg5g6dwV\...,2022-05-15 05:41:11+00:00


In [9]:
df.head()

,Text\r,Datetime
0,【123战法早8点播报-2022.5.3】\n#BTC 日线：空，小时线：多\n#ETH ...,2022-05-02 23:59:59+00:00
1,https://t.co/MeCEt7jfgS @DNC will be approving...,2022-05-02 23:59:45+00:00
2,4 hour top movers report #blockchain #crypto #...,2022-05-02 23:59:15+00:00
3,#Bitcoin Last Price $38512 #BTC 🚀\nDaily Indic...,2022-05-02 23:59:12+00:00
4,$RickMortyDoxx $RMD #RMDOXX #ETH #Ethereum #Ai...,2022-05-02 23:59:02+00:00


In [10]:
def tweet_to_words(tweet):
    text = tweet.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    words = text.split()
    words = [w for w in words if w not in stopwords.words("english")]
    words = [PorterStemmer().stem(w) for w in words]
    return words

In [11]:
cleantext=[]
for item in tqdm(df['Text\r']):
    words=tweet_to_words(item)
    cleantext+=[words]
df['cleaned']=cleantext
df

100%|██████████| 150015/150015 [21:50<00:00, 114.44it/s]


,Text\r,Datetime,cleaned
0,【123战法早8点播报-2022.5.3】\n#BTC 日线：空，小时线：多\n#ETH ...,2022-05-02 23:59:59+00:00,"[123, 8, 2022, 5, 3, btc, eth, btc38000, eth2780]"
1,https://t.co/MeCEt7jfgS @DNC will be approving...,2022-05-02 23:59:45+00:00,"[http, co, mecet7jfg, dnc, approv, 33, billion..."
2,4 hour top movers report #blockchain #crypto #...,2022-05-02 23:59:15+00:00,"[4, hour, top, mover, report, blockchain, cryp..."
3,#Bitcoin Last Price $38512 #BTC 🚀\nDaily Indic...,2022-05-02 23:59:12+00:00,"[bitcoin, last, price, 38512, btc, daili, indi..."
4,$RickMortyDoxx $RMD #RMDOXX #ETH #Ethereum #Ai...,2022-05-02 23:59:02+00:00,"[rickmortydoxx, rmd, rmdoxx, eth, ethereum, ai..."
...,...,...,...
150010,🙌good morning to the crowd🙌\n\n...and have a n...,2022-05-15 05:41:39+00:00,"[good, morn, crowd, nice, sunday, nft, eth, nf..."
150011,📣 Curated Coin Metrics every day 📣\n\n#ETH | ...,2022-05-15 05:41:27+00:00,"[curat, coin, metric, everi, day, eth, 2019, u..."
150012,Paris No. 3 💜 (1/1) .10eth\n\nI BELONG IN (TRA...,2022-05-15 05:41:14+00:00,"[pari, 3, 1, 1, 10eth, belong, travel, nft, co..."
150013,Woman by MaiPrisin\n\nhttps://t.co/GKcg5g6dwV\...,2022-05-15 05:41:11+00:00,"[woman, maiprisin, http, co, gkcg5g6dwv, found..."


In [12]:
def unlist(list):
    words=''
    for item in list:
        words+=item+' '
    return words

In [13]:
def compute_vader_scores(df, label):
    sid = SentimentIntensityAnalyzer()
    df["negative"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neg"])
    df["neutral"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neu"])
    df["positive"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["pos"])
    df["comp"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["compound"])
    return df

In [14]:
final_ethereum = compute_vader_scores(df,'cleaned')
final_ethereum

,Text\r,Datetime,cleaned,negative,neutral,positive,comp
0,【123战法早8点播报-2022.5.3】\n#BTC 日线：空，小时线：多\n#ETH ...,2022-05-02 23:59:59+00:00,"[123, 8, 2022, 5, 3, btc, eth, btc38000, eth2780]",0.000,1.000,0.000,0.0000
1,https://t.co/MeCEt7jfgS @DNC will be approving...,2022-05-02 23:59:45+00:00,"[http, co, mecet7jfg, dnc, approv, 33, billion...",0.099,0.901,0.000,-0.5106
2,4 hour top movers report #blockchain #crypto #...,2022-05-02 23:59:15+00:00,"[4, hour, top, mover, report, blockchain, cryp...",0.000,0.913,0.087,0.2023
3,#Bitcoin Last Price $38512 #BTC 🚀\nDaily Indic...,2022-05-02 23:59:12+00:00,"[bitcoin, last, price, 38512, btc, daili, indi...",0.053,0.947,0.000,-0.2960
4,$RickMortyDoxx $RMD #RMDOXX #ETH #Ethereum #Ai...,2022-05-02 23:59:02+00:00,"[rickmortydoxx, rmd, rmdoxx, eth, ethereum, ai...",0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...
150010,🙌good morning to the crowd🙌\n\n...and have a n...,2022-05-15 05:41:39+00:00,"[good, morn, crowd, nice, sunday, nft, eth, nf...",0.000,0.612,0.388,0.6908
150011,📣 Curated Coin Metrics every day 📣\n\n#ETH | ...,2022-05-15 05:41:27+00:00,"[curat, coin, metric, everi, day, eth, 2019, u...",0.000,1.000,0.000,0.0000
150012,Paris No. 3 💜 (1/1) .10eth\n\nI BELONG IN (TRA...,2022-05-15 05:41:14+00:00,"[pari, 3, 1, 1, 10eth, belong, travel, nft, co...",0.000,1.000,0.000,0.0000
150013,Woman by MaiPrisin\n\nhttps://t.co/GKcg5g6dwV\...,2022-05-15 05:41:11+00:00,"[woman, maiprisin, http, co, gkcg5g6dwv, found...",0.000,1.000,0.000,0.0000


In [ ]:
# print(len(date), len(final_bitcoin.cleaned),len(final_bitcoin.negative),len(final_bitcoin.comp),len(final_bitcoin.neutral))

In [15]:
# final_bitcoin.isnull().values.any()

final_ethereum[['Datetime','Text']] = final_ethereum.Datetime.str.split("+",expand=True)

final_ethereum
               


,Text\r,Datetime,cleaned,negative,neutral,positive,comp,Text
0,【123战法早8点播报-2022.5.3】\n#BTC 日线：空，小时线：多\n#ETH ...,2022-05-02 23:59:59,"[123, 8, 2022, 5, 3, btc, eth, btc38000, eth2780]",0.000,1.000,0.000,0.0000,00:00
1,https://t.co/MeCEt7jfgS @DNC will be approving...,2022-05-02 23:59:45,"[http, co, mecet7jfg, dnc, approv, 33, billion...",0.099,0.901,0.000,-0.5106,00:00
2,4 hour top movers report #blockchain #crypto #...,2022-05-02 23:59:15,"[4, hour, top, mover, report, blockchain, cryp...",0.000,0.913,0.087,0.2023,00:00
3,#Bitcoin Last Price $38512 #BTC 🚀\nDaily Indic...,2022-05-02 23:59:12,"[bitcoin, last, price, 38512, btc, daili, indi...",0.053,0.947,0.000,-0.2960,00:00
4,$RickMortyDoxx $RMD #RMDOXX #ETH #Ethereum #Ai...,2022-05-02 23:59:02,"[rickmortydoxx, rmd, rmdoxx, eth, ethereum, ai...",0.000,1.000,0.000,0.0000,00:00
...,...,...,...,...,...,...,...,...
150010,🙌good morning to the crowd🙌\n\n...and have a n...,2022-05-15 05:41:39,"[good, morn, crowd, nice, sunday, nft, eth, nf...",0.000,0.612,0.388,0.6908,00:00
150011,📣 Curated Coin Metrics every day 📣\n\n#ETH | ...,2022-05-15 05:41:27,"[curat, coin, metric, everi, day, eth, 2019, u...",0.000,1.000,0.000,0.0000,00:00
150012,Paris No. 3 💜 (1/1) .10eth\n\nI BELONG IN (TRA...,2022-05-15 05:41:14,"[pari, 3, 1, 1, 10eth, belong, travel, nft, co...",0.000,1.000,0.000,0.0000,00:00
150013,Woman by MaiPrisin\n\nhttps://t.co/GKcg5g6dwV\...,2022-05-15 05:41:11,"[woman, maiprisin, http, co, gkcg5g6dwv, found...",0.000,1.000,0.000,0.0000,00:00


In [16]:
final_ethereum = final_ethereum.filter(['Datetime', 'cleaned','negative','neutral','positive','comp'])
final_ethereum.head(40)

,Datetime,cleaned,negative,neutral,positive,comp
0,2022-05-02 23:59:59,"[123, 8, 2022, 5, 3, btc, eth, btc38000, eth2780]",0.000,1.000,0.000,0.0000
1,2022-05-02 23:59:45,"[http, co, mecet7jfg, dnc, approv, 33, billion...",0.099,0.901,0.000,-0.5106
2,2022-05-02 23:59:15,"[4, hour, top, mover, report, blockchain, cryp...",0.000,0.913,0.087,0.2023
3,2022-05-02 23:59:12,"[bitcoin, last, price, 38512, btc, daili, indi...",0.053,0.947,0.000,-0.2960
4,2022-05-02 23:59:02,"[rickmortydoxx, rmd, rmdoxx, eth, ethereum, ai...",0.000,1.000,0.000,0.0000
5,2022-05-02 23:58:55,"[bore, ape, crash, ethereum, transact, fee, we...",0.225,0.692,0.084,-0.5994
6,2022-05-02 23:58:42,"[tokenviz, biggest, crypto, 1, day, volum, bit...",0.000,1.000,0.000,0.0000
7,2022-05-02 23:58:31,"[vthousdt, bull, alert, 5x, volum, price, 0, 0...",0.000,0.901,0.099,0.2960
8,2022-05-02 23:58:24,"[eth, price, 2, 857, 38, right, compar, last, ...",0.080,0.920,0.000,-0.2732
9,2022-05-02 23:58:23,"[hi, thebinancenft, hold, http, co, ecpvpbxlic...",0.000,0.903,0.097,0.4404


In [19]:
final_ethereum.to_csv('../data/ethereum_cleaned.csv')
